In [1]:
import numpy as np

In [2]:
import pygame

SCREEN_WIDTH = 1000
SCREEN_HEIGHT = 200
FPS = 20

N = 200
L = 1.0
dx = L / (N - 1)
dt = 0.0001
a=1
lam = a * dt / (dx)**2
main_diagonal = np.ones(N-2)*(1+2*lam)
off_diagonal = np.ones(N-3) * (-lam)
A = np.diag(main_diagonal) + np.diag(off_diagonal,k=1) + np.diag(off_diagonal, k=-1)

u = np.zeros(N)

u[N//2 - 5 : N//2 + 5] = 5.0
#u[-30:] = 10.0
#u[:30]=5.0

def create_tridiag_matrix(main_val,off_val,size=N-2):
    main_diag = np.ones(size)*main_val
    off_diag = np.ones(size-1)*off_val
    A = np.diag(main_diag) + np.diag(off_diag,k=1) + np.diag(off_diag, k=-1)
    return A

A_c = create_tridiag_matrix(1+lam, -lam/2)
B_c = create_tridiag_matrix(1-lam, lam/2)

def step_heat_equation_crank_nicolson(u_input, dt, dx):
    u_new = np.zeros(N)
    rhs = B_c @ u_input[1:-1]
    u_new[1:-1] = np.linalg.solve(A_c, rhs)
    return u_new
    

def step_heat_equation_forward(u_input, dt, dx):
    u_new = np.zeros(N)
    u_new[1:-1] = a * dt*(u_input[2:] + u_input[:-2] - 2*u_input[1:-1])/((dx)**2) + u_input[1:-1]
    return u_new


def step_heat_equation_backward(u_input, dt, dx):
    u_new = np.zeros(N)
    #Step 2: calculate u_new
    u_new[1:-1] = np.linalg.solve(A,u_input[1:-1])
    return u_new


def step_heat_equation(u_input, dt, dx):
    return step_heat_equation_crank_nicolson(u_input, dt, dx)

pygame 2.6.1 (SDL 2.28.4, Python 3.12.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


C:\Users\Public\Documents\Fluid simulation\.venv\Lib\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


In [3]:
pygame.init()
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
clock = pygame.time.Clock()

def temperature_to_color(T):
    
    T = max(0.0, min(1.0, T))
    r = int(255 * T)
    b = int(255 * (1 - T))
    return (r, 0, b)

running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    u[:] = step_heat_equation(u, dt, dx)

    screen.fill((0, 0, 0))

    cell_width = SCREEN_WIDTH // N
    for i in range(N):
        color = temperature_to_color(u[i])
        pygame.draw.rect(
            screen,
            color,
            (i * cell_width, 0, cell_width + 1, SCREEN_HEIGHT)
        )

    pygame.display.flip()
    clock.tick(FPS)

pygame.quit()


In [4]:
np.ones(4)*(1+2*1)

array([3., 3., 3., 3.])